<a href="https://colab.research.google.com/github/DSE-Group7/5g-energy-consumption-modeling/blob/main/Notebooks/Kajaani/LightGBM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gc
import os
import itertools
import pickle
import re
import time

import warnings
warnings.filterwarnings('ignore')

from random import choice, choices
from functools import reduce
from tqdm import tqdm
from itertools import cycle

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
%matplotlib inline

from functools import reduce
from tqdm import tqdm
from itertools import cycle
from scipy import stats
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import ensemble
from sklearn import decomposition
from sklearn import tree

import lightgbm as lgb
import xgboost as xgb

pd.set_option("display.max_columns", None)

plt.style.use("ggplot")
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cell = pd.read_csv("/content/drive/MyDrive/competitions/input/CLdata.csv")
base = pd.read_csv("/content/drive/MyDrive/competitions/input/BSinfo.csv")
energy = pd.read_csv("/content/drive/MyDrive/competitions/input/ECdata.csv")

In [4]:
base.head()

,BS,CellName,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,B_0,Cell0,Type1,Mode2,365.0,20,4,6.875934
1,B_1,Cell0,Type2,Mode2,532.0,20,4,6.875934
2,B_2,Cell0,Type1,Mode2,365.0,20,4,6.875934
3,B_3,Cell0,Type2,Mode2,532.0,20,4,6.875934
4,B_4,Cell0,Type2,Mode2,532.0,20,4,6.875934


In [5]:
cell.head()

,Time,BS,CellName,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,ESMode6
0,1/1/2023 1:00,B_0,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0
1,1/1/2023 2:00,B_0,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0
2,1/1/2023 3:00,B_0,Cell0,0.193766,0.0,0.0,0.0,0,0.0,0.0
3,1/1/2023 4:00,B_0,Cell0,0.222383,0.0,0.0,0.0,0,0.0,0.0
4,1/1/2023 5:00,B_0,Cell0,0.175436,0.0,0.0,0.0,0,0.0,0.0


In [6]:
energy.head()

,Time,BS,Energy
0,1/1/2023 1:00,B_0,64.275037
1,1/1/2023 2:00,B_0,55.904335
2,1/1/2023 3:00,B_0,57.698057
3,1/1/2023 4:00,B_0,55.156951
4,1/1/2023 5:00,B_0,56.053812


In [7]:
# merge base and cell
base_cell = cell.merge(base, on=['BS','CellName'], how='left')
base_cell.head()

,Time,BS,CellName,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,ESMode6,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,1/1/2023 1:00,B_0,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
1,1/1/2023 2:00,B_0,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
2,1/1/2023 3:00,B_0,Cell0,0.193766,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
3,1/1/2023 4:00,B_0,Cell0,0.222383,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
4,1/1/2023 5:00,B_0,Cell0,0.175436,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934


In [8]:
data = energy.merge(base_cell, on=['Time','BS'], how='left')
data.head(10)

,Time,BS,Energy,CellName,load,ESMode1,ESMode2,ESMode3,ESMode4,ESMode5,ESMode6,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,1/1/2023 1:00,B_0,64.275037,Cell0,0.487936,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
1,1/1/2023 2:00,B_0,55.904335,Cell0,0.344468,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
2,1/1/2023 3:00,B_0,57.698057,Cell0,0.193766,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
3,1/1/2023 4:00,B_0,55.156951,Cell0,0.222383,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
4,1/1/2023 5:00,B_0,56.053812,Cell0,0.175436,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
5,1/1/2023 7:00,B_0,82.959641,Cell0,0.351585,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
6,1/1/2023 8:00,B_0,91.031390,Cell0,0.759298,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
7,1/1/2023 9:00,B_0,78.176383,Cell0,0.887798,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
8,1/1/2023 10:00,B_0,72.645740,Cell0,0.624745,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
9,1/1/2023 14:00,B_0,66.666667,Cell0,0.437926,0.0,0.0,0.0,0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934


In [9]:
data.shape

(98084, 17)

In [10]:
data.isnull().sum()

Time         0
BS           0
Energy       0
CellName     0
load         0
ESMode1      0
ESMode2      0
ESMode3      0
ESMode4      0
ESMode5      0
ESMode6      0
RUType       0
Mode         0
Frequency    0
Bandwidth    0
Antennas     0
TXpower      0
dtype: int64

In [11]:
# drop Time, ESMode3,ESMode4,ESMode5
data.drop(['ESMode3','ESMode4','ESMode5'], axis=1, inplace=True)
data.head()

,Time,BS,Energy,CellName,load,ESMode1,ESMode2,ESMode6,RUType,Mode,Frequency,Bandwidth,Antennas,TXpower
0,1/1/2023 1:00,B_0,64.275037,Cell0,0.487936,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
1,1/1/2023 2:00,B_0,55.904335,Cell0,0.344468,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
2,1/1/2023 3:00,B_0,57.698057,Cell0,0.193766,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
3,1/1/2023 4:00,B_0,55.156951,Cell0,0.222383,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934
4,1/1/2023 5:00,B_0,56.053812,Cell0,0.175436,0.0,0.0,0.0,Type1,Mode2,365.0,20,4,6.875934


In [12]:
df = data.pivot(
    index=['Time', 'BS'],
    columns=['CellName'],
    values=['load', 'ESMode1', 'ESMode2', 'ESMode6', 'Frequency',
       'Bandwidth', 'Antennas', 'TXpower'],
).reset_index()

df.columns = ['_'.join([str(i) for i in x]) for x in df.columns]
df.columns = df.columns.str.strip('_')
df = df.merge(base.groupby('BS')[['RUType', 'Mode']].first().reset_index(), on='BS', how='left')

In [13]:
df

,Time,BS,load_Cell0,load_Cell1,load_Cell2,load_Cell3,ESMode1_Cell0,ESMode1_Cell1,ESMode1_Cell2,ESMode1_Cell3,ESMode2_Cell0,ESMode2_Cell1,ESMode2_Cell2,ESMode2_Cell3,ESMode6_Cell0,ESMode6_Cell1,ESMode6_Cell2,ESMode6_Cell3,Frequency_Cell0,Frequency_Cell1,Frequency_Cell2,Frequency_Cell3,Bandwidth_Cell0,Bandwidth_Cell1,Bandwidth_Cell2,Bandwidth_Cell3,Antennas_Cell0,Antennas_Cell1,Antennas_Cell2,Antennas_Cell3,TXpower_Cell0,TXpower_Cell1,TXpower_Cell2,TXpower_Cell3,RUType,Mode
0,1/1/2023 10:00,B_0,0.624745,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,365.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,Type1,Mode2
1,1/1/2023 10:00,B_1,0.028090,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,532.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,Type2,Mode2
2,1/1/2023 10:00,B_10,0.020947,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,365.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,Type1,Mode2
3,1/1/2023 10:00,B_102,0.107020,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,189.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.427504,NaN,NaN,NaN,Type6,Mode2
4,1/1/2023 10:00,B_103,0.084820,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,189.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.427504,NaN,NaN,NaN,Type6,Mode2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92624,1/8/2023 0:00,B_788,0.137090,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,532.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,Type4,Mode2
92625,1/8/2023 0:00,B_789,0.321468,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,365.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,Type1,Mode2
92626,1/8/2023 0:00,B_793,0.095300,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,189.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.427504,NaN,NaN,NaN,Type6,Mode2
92627,1/8/2023 0:00,B_795,0.537426,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,365.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,Type1,Mode2


In [14]:
df.columns

Index(['Time', 'BS', 'load_Cell0', 'load_Cell1', 'load_Cell2', 'load_Cell3',
       'ESMode1_Cell0', 'ESMode1_Cell1', 'ESMode1_Cell2', 'ESMode1_Cell3',
       'ESMode2_Cell0', 'ESMode2_Cell1', 'ESMode2_Cell2', 'ESMode2_Cell3',
       'ESMode6_Cell0', 'ESMode6_Cell1', 'ESMode6_Cell2', 'ESMode6_Cell3',
       'Frequency_Cell0', 'Frequency_Cell1', 'Frequency_Cell2',
       'Frequency_Cell3', 'Bandwidth_Cell0', 'Bandwidth_Cell1',
       'Bandwidth_Cell2', 'Bandwidth_Cell3', 'Antennas_Cell0',
       'Antennas_Cell1', 'Antennas_Cell2', 'Antennas_Cell3', 'TXpower_Cell0',
       'TXpower_Cell1', 'TXpower_Cell2', 'TXpower_Cell3', 'RUType', 'Mode'],
      dtype='object')

In [15]:
df.describe()

,load_Cell0,load_Cell1,load_Cell2,load_Cell3,ESMode1_Cell0,ESMode1_Cell1,ESMode1_Cell2,ESMode1_Cell3,ESMode2_Cell0,ESMode2_Cell1,ESMode2_Cell2,ESMode2_Cell3,ESMode6_Cell0,ESMode6_Cell1,ESMode6_Cell2,ESMode6_Cell3,Frequency_Cell0,Frequency_Cell1,Frequency_Cell2,Frequency_Cell3,Bandwidth_Cell0,Bandwidth_Cell1,Bandwidth_Cell2,Bandwidth_Cell3,Antennas_Cell0,Antennas_Cell1,Antennas_Cell2,Antennas_Cell3,TXpower_Cell0,TXpower_Cell1,TXpower_Cell2,TXpower_Cell3
count,92629.000000,5375.000000,40.000000,40.000000,92629.000000,5375.000000,40.0,40.0,92629.000000,5375.000000,40.0,40.0,92629.000000,5375.000000,40.0,40.0,92629.000000,5375.000000,40.00,40.00,92629.000000,5375.000000,40.0,40.0,92629.000000,5375.00000,40.0,40.0,92629.000000,5375.000000,40.000000,40.000000
mean,0.249718,0.111894,0.072556,0.081545,0.041080,0.003941,0.0,0.0,0.039769,0.003106,0.0,0.0,0.009672,0.387457,0.0,0.0,368.460920,377.050910,426.98,426.98,16.541094,6.261581,2.0,2.0,2.187242,2.17600,2.0,2.0,6.716416,7.026157,6.877429,6.877429
std,0.240476,0.090979,0.024943,0.037832,0.192014,0.058264,0.0,0.0,0.189299,0.051156,0.0,0.0,0.081137,0.396055,0.0,0.0,138.170799,99.108267,0.00,0.00,5.056101,6.181299,0.0,0.0,2.088378,0.57447,0.0,0.0,0.290863,0.385209,0.000000,0.000000
min,0.000000,0.008190,0.045942,0.045904,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,155.600000,155.600000,426.98,426.98,2.000000,2.000000,2.0,2.0,1.000000,1.00000,2.0,2.0,5.381166,6.427504,6.877429,6.877429
25%,0.056702,0.052356,0.056452,0.056832,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,189.000000,365.000000,426.98,426.98,10.000000,2.000000,2.0,2.0,2.000000,2.00000,2.0,2.0,6.427504,6.427504,6.877429,6.877429
50%,0.167217,0.080615,0.067933,0.067856,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.265585,0.0,0.0,365.000000,426.980000,426.98,426.98,20.000000,2.000000,2.0,2.0,2.000000,2.00000,2.0,2.0,6.875934,7.325859,6.877429,6.877429
75%,0.372600,0.139284,0.081745,0.095495,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.804151,0.0,0.0,532.000000,426.980000,426.98,426.98,20.000000,10.000000,2.0,2.0,2.000000,2.00000,2.0,2.0,6.875934,7.325859,6.877429,6.877429
max,0.993957,0.891770,0.174846,0.196519,1.000000,1.000000,0.0,0.0,1.000000,1.000000,0.0,0.0,0.931032,0.927924,0.0,0.0,979.998000,697.002000,426.98,426.98,20.000000,20.000000,2.0,2.0,64.000000,4.00000,2.0,2.0,8.375336,7.877728,6.877429,6.877429


In [16]:
df = df.merge(energy, on=['Time', 'BS'], how='left')
df

,Time,BS,load_Cell0,load_Cell1,load_Cell2,load_Cell3,ESMode1_Cell0,ESMode1_Cell1,ESMode1_Cell2,ESMode1_Cell3,ESMode2_Cell0,ESMode2_Cell1,ESMode2_Cell2,ESMode2_Cell3,ESMode6_Cell0,ESMode6_Cell1,ESMode6_Cell2,ESMode6_Cell3,Frequency_Cell0,Frequency_Cell1,Frequency_Cell2,Frequency_Cell3,Bandwidth_Cell0,Bandwidth_Cell1,Bandwidth_Cell2,Bandwidth_Cell3,Antennas_Cell0,Antennas_Cell1,Antennas_Cell2,Antennas_Cell3,TXpower_Cell0,TXpower_Cell1,TXpower_Cell2,TXpower_Cell3,RUType,Mode,Energy
0,1/1/2023 10:00,B_0,0.624745,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,365.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,Type1,Mode2,72.645740
1,1/1/2023 10:00,B_1,0.028090,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,532.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,Type2,Mode2,22.571001
2,1/1/2023 10:00,B_10,0.020947,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,365.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,Type1,Mode2,31.240658
3,1/1/2023 10:00,B_102,0.107020,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,189.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.427504,NaN,NaN,NaN,Type6,Mode2,19.282511
4,1/1/2023 10:00,B_103,0.084820,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,189.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.427504,NaN,NaN,NaN,Type6,Mode2,18.236173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92624,1/8/2023 0:00,B_788,0.137090,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,532.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,Type4,Mode2,22.421525
92625,1/8/2023 0:00,B_789,0.321468,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,365.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,Type1,Mode2,42.600897
92626,1/8/2023 0:00,B_793,0.095300,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,189.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.427504,NaN,NaN,NaN,Type6,Mode2,18.236173
92627,1/8/2023 0:00,B_795,0.537426,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,365.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,Type1,Mode2,47.234679


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92629 entries, 0 to 92628
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Time             92629 non-null  object 
 1   BS               92629 non-null  object 
 2   load_Cell0       92629 non-null  float64
 3   load_Cell1       5375 non-null   float64
 4   load_Cell2       40 non-null     float64
 5   load_Cell3       40 non-null     float64
 6   ESMode1_Cell0    92629 non-null  float64
 7   ESMode1_Cell1    5375 non-null   float64
 8   ESMode1_Cell2    40 non-null     float64
 9   ESMode1_Cell3    40 non-null     float64
 10  ESMode2_Cell0    92629 non-null  float64
 11  ESMode2_Cell1    5375 non-null   float64
 12  ESMode2_Cell2    40 non-null     float64
 13  ESMode2_Cell3    40 non-null     float64
 14  ESMode6_Cell0    92629 non-null  float64
 15  ESMode6_Cell1    5375 non-null   float64
 16  ESMode6_Cell2    40 non-null     float64
 17  ESMode6_Cell

In [18]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00


In [19]:
MLFLOW_TRACKING_URI="https://dagshub.com/samithkavishke/5g-energy-consumption-modeling.mlflow"
MLFLOW_TRACKING_USERNAME="Kajaani-Balabavan"
MLFLOW_TRACKING_PASSWORD="76352022f25ef66bfc854478c2caf765d3ab72c4"

In [20]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("Competition")

<Experiment: artifact_location='mlflow-artifacts:/05e4fbc588934e979ba8497f14f54669', creation_time=1698146260875, experiment_id='0', last_update_time=1698146260875, lifecycle_stage='active', name='Competition', tags={}>

In [21]:
os.environ['MLFLOW_TRACKING_USERNAME'] = MLFLOW_TRACKING_USERNAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = MLFLOW_TRACKING_PASSWORD

In [22]:
df['BS'] = df['BS'].str.replace('B_', '').astype(int)
df['RUType'] = df['RUType'].str.replace('Type', '').astype(int)
df['Mode'] = df['Mode'].str.replace('Mode', '').astype(int)
df['Time'] = pd.to_datetime(df['Time'])
df['hour'] = df['Time'].dt.hour
# df = pd.get_dummies(df, columns=['RUType',  'Mode', 'hour'])
df.drop(['Time'], axis=1, inplace=True)
df.head()

,BS,load_Cell0,load_Cell1,load_Cell2,load_Cell3,ESMode1_Cell0,ESMode1_Cell1,ESMode1_Cell2,ESMode1_Cell3,ESMode2_Cell0,ESMode2_Cell1,ESMode2_Cell2,ESMode2_Cell3,ESMode6_Cell0,ESMode6_Cell1,ESMode6_Cell2,ESMode6_Cell3,Frequency_Cell0,Frequency_Cell1,Frequency_Cell2,Frequency_Cell3,Bandwidth_Cell0,Bandwidth_Cell1,Bandwidth_Cell2,Bandwidth_Cell3,Antennas_Cell0,Antennas_Cell1,Antennas_Cell2,Antennas_Cell3,TXpower_Cell0,TXpower_Cell1,TXpower_Cell2,TXpower_Cell3,RUType,Mode,Energy,hour
0,0,0.624745,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,365.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,1,2,72.645740,10
1,1,0.028090,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,532.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,2,2,22.571001,10
2,10,0.020947,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,365.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.875934,NaN,NaN,NaN,1,2,31.240658,10
3,102,0.107020,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,189.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.427504,NaN,NaN,NaN,6,2,19.282511,10
4,103,0.084820,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,189.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.427504,NaN,NaN,NaN,6,2,18.236173,10


In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import mlflow
import os
import pickle

# Define your features (X) and target variable (y)
X = df.drop("Energy", axis=1)  # Features (excluding the "Energy" column)
y = df["Energy"]  # Target variable

# Define your LightGBM model
params = {
    'objective': 'regression',
        'boosting_type': 'gbdt',
        'learning_rate': 0.1,
        'n_jobs': -1,
        'max_depth' : -1,
        'metric': 'mae',
        'num_boost_round': 10000,
}

# Create a KFold cross-validation object with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store the scores at each fold
mae_scores = []
rmse_scores = []
r2_scores = []

# Perform cross-validation and get the scores at each fold
for i, (train_idx, valid_idx) in enumerate(kf.split(X)):
    X_train_fold, X_valid_fold = X.iloc[train_idx], X.iloc[valid_idx]
    y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]

    # Define the LightGBM model
    model = lgb.LGBMRegressor(**params)

    # Fit the model
    # model.fit(X_train_fold, y_train_fold, eval_set=[(X_valid_fold, y_valid_fold)], verbose=False)

    early_stopping_callback = lgb.early_stopping(100, first_metric_only=True, verbose=False)
    verbose_callback = lgb.log_evaluation(100)

    model.fit(
        X_train_fold, y_train_fold, eval_set=[(X_valid_fold, y_valid_fold)],
        callbacks=[early_stopping_callback, verbose_callback],
    )
    # Make predictions
    y_pred = model.predict(X_valid_fold, num_iteration=model.best_iteration_)

    # Calculate evaluation metrics
    mae = mean_absolute_error(y_valid_fold, y_pred)
    rmse = np.sqrt(mean_squared_error(y_valid_fold, y_pred))
    r2 = r2_score(y_valid_fold, y_pred)

    # Append scores to the lists
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

    # Print the results for each fold
    print(f"Fold {i + 1}:")
    print(f"MAE: {mae}")
    print(f"RMSE: {rmse}")
    print(f"R-squared: {r2}")

# Calculate the means of the scores
mean_mae = np.mean(mae_scores)
mean_rmse = np.mean(rmse_scores)
mean_r2 = np.mean(r2_scores)

# Print and log the overall results
print("=*" * 50)
print("Overall Results:")
print(f"Mean Absolute Error (MAE): {mean_mae}")
print(f"Root Mean Squared Error (RMSE): {mean_rmse}")
print(f"R-squared (R2) Score: {mean_r2}")

# Logging parameters and metrics using MLflow
with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metric("MAE", mean_mae)
    mlflow.log_metric("RMSE", mean_rmse)
    mlflow.log_metric("R2", mean_r2)

    # Save the LightGBM model as a pickle file
    model_name = "lightgbm_model.pkl"
    with open(model_name, 'wb') as fout:
        pickle.dump(model, fout)

    # Log the model as an artifact
    mlflow.log_artifact(model_name)


[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017790 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1919
[LightGBM] [Info] Number of data points in the train set: 74103, number of used features: 36
[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Info] Start training from score 28.136636
[100]	valid_0's l1: 1.95393
[200]	valid_0's l1: 1.7188
[300]	valid_0's l1: 1.61892
[400]	valid_0's l1: 1.54547
[500]	valid_0's l1: 1.50736
[600]	valid_0's l1: 1.48333
[700]	valid_0's l1: 1.46083
[800]	valid_0's l1: 1.44431
[900]	valid_0's l1: 1.43103
[1000]	valid_0's l1: 1.41846
[1100]	valid_0's l1: 1.40959
[1200]	valid_0's l1: 1.40243
[1300